# Complete Simulation Chain for Visibility Calculation

## Read in Config

In [16]:
import vipy.simulation.utils as ut
import vipy.layouts.layouts as layouts
import astropy.constants as const
from astropy import units as un
import time as t
import numpy as np

In [17]:
# rc = ut.read_config('/net/nfshome/home/sfroese/vipy/config/default.toml')
rc = ut.read_config("../config/test.toml")
array_layout = layouts.get_array_layout('eht')

src_crd = rc['src_coord']
wave1 = const.c/((float(rc['channel'].split(':')[0])-float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter
wave2 = const.c/((float(rc['channel'].split(':')[0])+float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter

## Generate (l,m)-plane / FOV

In [18]:
import vipy.simulation.scan as scan

In [19]:
grid = scan.create_bgrid(rc['fov_size'], 256, src_crd)
lm = scan.lm(grid, src_crd)

## Calculate Start and Stop times for every measurement

In [20]:
time = ut.calc_time_steps(rc)
print(time)

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:00:20.000'
 '2016:095:00:20:00.000' '2016:095:00:20:10.000' '2016:095:00:20:20.000'
 '2016:095:00:40:00.000' '2016:095:00:40:10.000' '2016:095:00:40:20.000'
 '2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:00:20.000'
 '2016:095:01:20:00.000' '2016:095:01:20:10.000' '2016:095:01:20:20.000']


## Calculate Baselines for one measurement

In [21]:
baselines = scan.get_baselines(src_crd, time[0:3], array_layout)

In [22]:
st1,st2 = scan.get_valid_baselines(baselines,28)
st1.shape

(24,)

## Calculate Mueller/Jones matrices

In [23]:
import numpy as np
import torch

In [24]:
%%time
start = t.time()
torch.set_num_threads(8)
JJ_f1 = scan.getJones(lm, baselines, wave1, time[0:3], src_crd, array_layout)
JJ_f2 = scan.getJones(lm, baselines, wave2, time[0:3], src_crd, array_layout)

CPU times: user 3.03 s, sys: 871 ms, total: 3.9 s
Wall time: 677 ms


## Open Source Image and compute Stokes vector

In [25]:
from astropy.io import fits
import matplotlib.pyplot as plt
import torch

In [26]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))
# plt.imshow((I[...,1]+I[...,2]).real)
# plt.colorbar()

## Integration

In [27]:
%%time
delta_t = rc['corr_int_time']
delta_f = float(rc['channel'].split(':')[1])*10**6
delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
print(delta_l)

integral = scan.integrate(JJ_f1, JJ_f2, I, 2, delta_t, delta_f, delta_l, delta_m)
integral.shape
end = t.time()
print(end - start)

0.0001830982550547064
3.151320457458496
CPU times: user 985 ms, sys: 396 ms, total: 1.38 s
Wall time: 180 ms


In [28]:
integral.shape

torch.Size([12, 4])

In [29]:
integral

tensor([[ 1.5361e-04+1.9068e-04j,  1.1414e-04+1.4162e-04j,
          7.3597e-05+9.1274e-05j,  5.4686e-05+6.7790e-05j],
        [-2.8102e-05-3.3827e-05j, -5.9045e-06-7.1092e-06j,
         -2.1364e-04-2.5706e-04j, -4.4891e-05-5.4023e-05j],
        [-2.5165e-04-3.9236e-05j, -1.8750e-04-2.9215e-05j,
         -1.9086e-03-2.9781e-04j, -1.4220e-03-2.2175e-04j],
        [-4.0103e-05-3.6748e-05j, -1.9255e-05-1.7631e-05j,
         -3.0474e-04-2.7937e-04j, -1.4632e-04-1.3404e-04j],
        [ 8.5381e-04+1.6212e-03j,  1.7994e-04+3.4074e-04j,
          6.3850e-04+1.2062e-03j,  1.3456e-04+2.5352e-04j],
        [-5.4484e-04-4.4131e-04j, -1.1451e-04-9.2606e-05j,
         -2.6166e-04-2.1069e-04j, -5.4996e-05-4.4211e-05j],
        [ 5.9552e-04-4.8154e-04j,  4.4461e-04-3.5925e-04j,
          2.8795e-04-2.3250e-04j,  2.1498e-04-1.7346e-04j],
        [-4.0878e-05+8.8357e-05j, -8.6068e-06+1.8604e-05j,
         -3.0934e-04+6.6865e-04j, -6.5133e-05+1.4079e-04j],
        [-2.7928e-04+5.8789e-07j, -2.0811e-04+4.

## All scan loop

In [30]:
from tqdm import tqdm

In [31]:
torch.set_num_threads(12)

In [223]:
from dataclasses import dataclass
# stokes, IFs?

@dataclass
class Visibilities:
    value: np.array(complex)
    num: [int]
    scan: [int]
    base_num: [int]
    u: [float]
    v: [float]
    w: [float]
    date: [float]
    _date: [float]

    def __getitem__(self, i):
        baseline = Vis(
            self.value[i],
            self.num[i],
            self.scan[i],
            self.base_num[i],
            self.u[i],
            self.v[i],
            self.w[i],
            self.date[i],
            self._date[i],
        )
        return baseline

    def add(self, visibilities):
        self.value = np.concatenate([self.value, visibilities.value])
        self.num = np.concatenate([self.num, visibilities.num])
        self.scan = np.concatenate([self.scan, visibilities.scan])
        self.base_num = np.concatenate([self.base_num, visibilities.base_num])
        self.u = np.concatenate([self.u, visibilities.u])
        self.v = np.concatenate([self.v, visibilities.v])
        self.w = np.concatenate([self.w, visibilities.w])
        self.date = np.concatenate([self.date, visibilities.date])
        self._date = np.concatenate([self._date, visibilities._date])
        
    def del_vis(self, i):
        self.value = np.delete(self.value, i, axis=0)
        self.num = np.delete(self.num, i, axis=0)
        self.scan = np.delete(self.scan, i, axis=0)
        self.base_num = np.delete(self.base_num, i, axis=0)
        self.u = np.delete(self.u, i, axis=0)
        self.v = np.delete(self.v, i, axis=0)
        self.w = np.delete(self.w, i, axis=0)
        self.date = np.delete(self.date, i, axis=0)
        self._date = np.delete(self._date, i, axis=0)


@dataclass
class Vis:
    value: complex
    num: int
    scan: int
    base_num: int
    u: float
    v: float
    w: float
    date: float
    _date: float

In [224]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))

In [246]:
from vipy.simulation.scan import get_valid_baselines
visibilities = Visibilities(np.empty((1, 4), dtype="complex"), [], [], [], [], [], [], [], [])
vis_num = np.zeros(1)
#i in total number of scans
for i in tqdm(range(2)):
    t = time[i*3:(i+1)*3]
    baselines = scan.get_baselines(src_crd, t, array_layout)
    
    valid = baselines.valid.reshape(-1, 28)
    mask = np.array(valid[:-1]).astype(bool) & np.array(valid[1:]).astype(bool)
    u = baselines.u.reshape(-1, 28)
    v = baselines.v.reshape(-1, 28)
    w = baselines.w.reshape(-1, 28)
    u_valid = u[:-1][mask]
    v_valid = v[:-1][mask]
    w_valid = w[:-1][mask]
    date = np.repeat(t[:-1].mjd.reshape(-1, 1), 28, axis=1)[mask]
    _date = np.zeros(len(u_valid))
    print(_date.shape)
    
    
    JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
    if JJ_f1.shape[0] == 1:
        continue
    JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)

    delta_t = rc['corr_int_time']
    delta_f = float(rc['channel'].split(':')[1])*10**6
    delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
    delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
    
    vis_num = np.arange(JJ_f1.shape[2]//2) + 1 + vis_num.max()

    vis = Visibilities(
        scan.integrate(JJ_f1, JJ_f2, I, 2, delta_t, delta_f, delta_l, delta_m),
        vis_num,
        np.repeat(i+1, len(vis_num)),
        np.array([baselines[i].baselineNum() for i in range(len(vis_num))]),
        u_valid,
        v_valid,
        w_valid,
        date,
        _date,      
    )
    
    visibilities.add(vis)

visibilities.del_vis(0)


  0%|          | 0/2 [00:00<?, ?it/s]

(12,)


 50%|█████     | 1/2 [00:00<00:00,  1.23it/s]

(12,)


100%|██████████| 2/2 [00:01<00:00,  1.24it/s]


In [249]:
visibilities.u.shape

(23,)

In [233]:
a = [1, 2, 3]

In [201]:
del a[0]
a

[2, 3]

In [66]:
baselines[0]

Baseline(name='PV-PdBI', st1=Station(st_num=4, name='PV', x=5088967.9, y=-301681.6, z=3825015.8, diam=30.0, el_low=15, el_high=85, sefd=2900, altitude=2850), st2=Station(st_num=5, name='PdBI', x=4523998.4, y=468045.24, z=4460309.76, diam=7.0, el_low=15, el_high=85, sefd=1600, altitude=2550), u=<Quantity -659343.66384915>, v=<Quantity 722159.25615713>, w=<Quantity -599183.02833747>, valid=1.0)

In [19]:
integral2 = integral2.reshape(-1, 4)
integral2.shape

(76, 4)

In [20]:
integral2

array([[ 1.53612912e-04+1.90676858e-04j,  1.14141964e-04+1.41617721e-04j,
         7.35971100e-05+9.12743597e-05j,  5.46863521e-05+6.77900933e-05j],
       [-2.81016866e-05-3.38269830e-05j, -5.90445952e-06-7.10919705e-06j,
        -2.13643069e-04-2.57056346e-04j, -4.48905846e-05-5.40232160e-05j],
       [-2.51645339e-04-3.92361397e-05j, -1.87497392e-04-2.92152935e-05j,
        -1.90856773e-03-2.97810937e-04j, -1.42204827e-03-2.21749209e-04j],
       [-4.01032835e-05-3.67480763e-05j, -1.92551676e-05-1.76313124e-05j,
        -3.04738211e-04-2.79365748e-04j, -1.46316355e-04-1.34036772e-04j],
       [ 8.53807532e-04+1.62116446e-03j,  1.79940639e-04+3.40743147e-04j,
         6.38499054e-04+1.20616109e-03j,  1.34562658e-04+2.53517201e-04j],
       [-5.44844334e-04-4.41314528e-04j, -1.14514940e-04-9.26056523e-05j,
        -2.61663928e-04-2.10690580e-04j, -5.49958281e-05-4.42107643e-05j],
       [ 5.95518488e-04-4.81535555e-04j,  4.44610589e-04-3.59247454e-04j,
         2.87945023e-04-2.325031

In [6]:
import import_ipynb
from fits_tests import create_vis_hdu

In [9]:
hdu_vis = create_vis_hdu(data, conf)

NameError: name 'data' is not defined

In [86]:
a = np.ones((12, 4))
b = np.zeros((14, 4))

In [110]:
np.concatenate((a, b))

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [115]:
c = np.empty((1, 4))
c

array([[4.24399158e-313, 0.00000000e+000, 4.24399158e-313,
        4.94065646e-323]])

In [130]:
np.concatenate((a, c))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 4 and the array at index 1 has size 0

In [152]:
c = np.empty((1, 4), dtype="complex")

In [215]:
np.delete(b, 0, axis=0).shape

(13, 4)

In [147]:
d = np.array([).shape

In [149]:
d.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [154]:
c.shape

(1, 4)